In [50]:
import numpy as np
import pandas as pd
import time
import warnings
from sklearn import linear_model 
from sklearn import cross_validation

In [51]:
filename = 'test.csv'
df = pd.read_csv(filename)
df.columns = ['Id', 'ints']
df['ints_list'] = df.ints.apply(lambda x: x.split(','))
df['ints_len'] = df.ints_list.apply(lambda x: len(x))
df['last'] = df.ints_list.apply(lambda x: int(x[-1]))
df.drop('ints', axis=1, inplace=True)
print('Longest:', df.ints_len.max())
longest = df.ints_len.max() + 1
df.head()

Longest: 347


,Id,ints_list,ints_len,last
0,1,"[1, 0, 0, 2, 24, 552, 21280, 103760, 70299264,...",11,587159944704
1,2,"[1, 1, 5, 11, 35, 93, 269, 747, 2115, 5933, 16...",27,257733967693
2,4,"[0, 1, 101, 2, 15, 102, 73, 3, 40, 16, 47, 103...",51,158
3,5,"[1, 4, 14, 23, 42, 33, 35, 34, 63, 66, 87, 116...",50,406
4,6,"[1, 1, 2, 5, 4, 2, 6, 13, 11, 4, 10, 10, 12, 6...",70,24


In [52]:
df_train = pd.DataFrame()
df_train['x'] = np.arange(longest)
df_train['x0'] = 1
df_train['x2'] = df_train.x ** 2
df_train['x3'] = df_train.x ** 3
df_train['x4'] = df_train.x ** 4
df_train['sqrt'] = df_train.x ** .5
df_train['exp'] = np.exp(df_train.x)
df_train['odd'] = df_train.x % 2
df_train['log'] = np.log(df_train.x)
df_train['sin'] = np.sin(df_train.x)
df_train['cos'] = np.cos(df_train.x)
df_train.tail()

,x,x0,x2,x3,x4,sqrt,exp,odd,log,sin,cos
343,343,1,117649,40353607,13841287201,18.520259,9.183480e+148,1,5.837730,-0.536598,-0.843838
344,344,1,118336,40707584,14003408896,18.547237,2.496329e+149,0,5.840642,-0.999990,-0.004396
345,345,1,119025,41063625,14166950625,18.574176,6.785725e+149,1,5.843544,-0.543996,0.839088
346,346,1,119716,41421736,14331920656,18.601075,1.844551e+150,0,5.846439,0.412146,0.911118
347,347,1,120409,41781923,14498327281,18.627936,5.014010e+150,1,5.849325,0.989363,0.145470


In [53]:
cols = [col for col in df_train.columns if col != 'res']
cols

['x', 'x0', 'x2', 'x3', 'x4', 'sqrt', 'exp', 'odd', 'log', 'sin', 'cos']

In [54]:
# Check recursion

def get_matrix(seq, order):
    A = []
    for i in range(order + 1):
        s = [1] + seq[i:i+order]
        A = s if i == 0 else np.vstack([A, s]) 
    b = seq[order:2*order+1]
    return A, b

def check_recursion(seq):
    p = len(seq) - 1
    n = int(p/2 -1)
    try:
        A, b = get_matrix(seq, n)
        if  np.linalg.matrix_rank(A) - 1 < n:
            n = np.linalg.matrix_rank(A) - 1
            A, b = get_matrix(seq, n)
        w = np.linalg.solve(A, b)    
        # check
        feat_check_A = [1] + seq[n+1:2*n+1]
        feat_check_b = seq[2*n+1]
        if np.dot(feat_check_A, w) == feat_check_b:        
            feat_pred_A = [1] + seq[p-n+1:]
            predict = np.dot(feat_pred_A, w)
            return int(predict)
        else: return np.nan        
    except:
        return np.nan

In [55]:
# check some recursions
import time
st = time.time()
def get_diffs(x):
    return [x[i+1] - x[i] for i in range(len(x) - 1)] if len(x) > 1 else x    

df['ints_list'] = df.ints_list.apply(lambda x: list(map(int, x)))
df['diffs'] = df.ints_list.apply(get_diffs)
df['plus1'] = df.ints_list.apply(lambda x: [i+1 for i in x])
df['minus1'] = df.ints_list.apply(lambda x: [i-1 for i in x])
print('time: %.2f min' % ((time.time() - st) / 60))

time: 0.10 min


In [56]:
st = time.time()
#print(df.ix[0])
df['recursion'] = df['Id'].map(pd.read_csv('test_recursion_result.csv', encoding='utf-8-sig').set_index('Id').recursion)
df['rec_diffs'] = df.apply(lambda row: check_recursion(row.diffs) + row['last'], axis=1)
df['rec_plus1'] = df.plus1.apply(lambda x: check_recursion(x) - 1)
df['rec_minus1'] = df.minus1.apply(lambda x: check_recursion(x) + 1)
print('time: %.2f min' % ((time.time() - st) / 60))

time: 4.27 min


In [57]:
print(df[df.recursion.notnull()].Id.count() / len(df.index))
df.recursion.fillna(df.rec_diffs, inplace=True)
df.recursion.fillna(df.rec_plus1, inplace=True)
df.recursion.fillna(df.rec_minus1, inplace=True)
print(df[df.recursion.notnull()].Id.count() / len(df.index))
df[['Id', 'recursion']].to_csv('test_full_recursion_result.csv', encoding='utf-8-sig')

0.0924678290658
0.126241820018


In [59]:
st = time.time()

# check recursion and write to file

#df['recursion'] = df.ints_list.apply(lambda x: check_recursion(list(map(int, x))))
#print('Recursions checked. Time elapsed: %.2f min' % ((time.time() - st) / 60))
#print('Recursion detected for %.4f sequences' % (df[df.recursion.isnull() == False].Id.count() / df.Id.count()))
#df[['Id', 'recursion']].to_csv('test_recursion_result.csv', index=False)
#pd.read_csv('test_recursion_result.csv', encoding='utf-8-sig')

# get recursion result from file
df['recursion'] = df['Id'].map(pd.read_csv('test_full_recursion_result.csv', encoding='utf-8-sig').set_index('Id').recursion)
df.head()

,Id,ints_list,ints_len,last,diffs,plus1,minus1,recursion,rec_diffs,rec_plus1,rec_minus1
0,1,"[1, 0, 0, 2, 24, 552, 21280, 103760, 70299264,...",11,587159944704,"[-1, 0, 2, 22, 528, 20728, 82480, 70195504, 57...","[2, 1, 1, 3, 25, 553, 21281, 103761, 70299265,...","[0, -1, -1, 1, 23, 551, 21279, 103759, 7029926...",NaN,NaN,NaN,NaN
1,2,"[1, 1, 5, 11, 35, 93, 269, 747, 2115, 5933, 16...",27,257733967693,"[0, 4, 6, 24, 58, 176, 478, 1368, 3818, 10784,...","[2, 2, 6, 12, 36, 94, 270, 748, 2116, 5934, 16...","[0, 0, 4, 10, 34, 92, 268, 746, 2114, 5932, 16...",7.251620e+11,725161963867,725161963867,NaN
2,4,"[0, 1, 101, 2, 15, 102, 73, 3, 40, 16, 47, 103...",51,158,"[1, 100, -99, 13, 87, -29, -70, 37, -24, 31, 5...","[1, 2, 102, 3, 16, 103, 74, 4, 41, 17, 48, 104...","[-1, 0, 100, 1, 14, 101, 72, 2, 39, 15, 46, 10...",NaN,NaN,NaN,NaN
3,5,"[1, 4, 14, 23, 42, 33, 35, 34, 63, 66, 87, 116...",50,406,"[3, 10, 9, 19, -9, 2, -1, 29, 3, 21, 29, -32, ...","[2, 5, 15, 24, 43, 34, 36, 35, 64, 67, 88, 117...","[0, 3, 13, 22, 41, 32, 34, 33, 62, 65, 86, 115...",NaN,NaN,NaN,NaN
4,6,"[1, 1, 2, 5, 4, 2, 6, 13, 11, 4, 10, 10, 12, 6...",70,24,"[0, 1, 3, -1, -2, 4, 7, -2, -7, 6, 0, 2, -6, 2...","[2, 2, 3, 6, 5, 3, 7, 14, 12, 5, 11, 11, 13, 7...","[0, 0, 1, 4, 3, 1, 5, 12, 10, 3, 9, 9, 11, 5, ...",NaN,NaN,NaN,NaN


In [61]:
preds = []
longest = df.ints_len.max() + 1
alphas = [10 ** x for x in range(-4, 4)]

#cnt = 5
cnt = len(df.index)

#nrows, ncols = 4, 4
#cnt = nrows * ncols
#fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(18,18))
#sns.set(style='whitegrid', context='notebook')

st = time.time()
for i in range(cnt):
    if (i % 2000 == 0) & (i != 0):
        print('Sequences: %d of %d (%.2f%% done), time elapsed: %.2f min, estimated time: %.2f min' 
              % (i, cnt, 100 * i / cnt, (time.time() - st) / 60, (time.time() - st) * cnt / (60 * i)))        
    #print(i)
    df1 = df.ix[i]
    if np.isnan(df1.recursion): # do linear regression only if no recursion detected
        df_curr = df_train[:df1.ints_len + 1].copy(deep=True)       
        df_curr['res'] = df1.ints_list + [np.nan]
        df_curr['prev'] = df_curr.res.shift(1)
        ints_ser = pd.Series(list(map(int, df1.ints_list)))
        default_value = df1.ints_list[-1]
        #def_func = np.mean(ints_ser)
        try:
            freqs = ints_ser[ints_ser < np.iinfo(np.int64).max].value_counts() 
            if len(freqs.index) > 2:
                freq, next_freq = freqs.iloc[0], freqs.iloc[1]
                default_value = freqs.idxmax() if freq / next_freq >= 2 else default_value            
        except OverflowError: default_value

        cols = [col for col in df_curr.columns if col != 'res']    
        df_curr = df_curr[1:].reset_index()

        if len(df_curr.index) > 1:
            X_train = df_curr[:-1][cols]
            y_train = df_curr[:-1].res          
            X_test = df_curr[-1:][cols]

            #lr = linear_model.LinearRegression()
            #lr.fit(X_train, y_train)
            #predict = lr.predict(X_test)[0]  
            #print('predict after LR:', predict)  

            with warnings.catch_warnings():
                warnings.simplefilter("error")            
                try:
                    lr_rigdeCV = linear_model.RidgeCV(alphas=alphas)
                    lr_rigdeCV.fit(X_train, y_train)
                    best_alpha = lr_rigdeCV.alpha_
                except: best_alpha = 0.1

                try:
                    lr_ridge = linear_model.Ridge(alpha=best_alpha)
                    lr_ridge.fit(X_train, y_train)
                    predict = lr_ridge.predict(X_test)[0]
                except: predict = default_value
                #print('predict after ridge LR:', predict)

                if len(X_train.index) > 5:
                    try:
                        kf = cross_validation.KFold(len(X_train.index), n_folds=5, shuffle=True)
                        scores = cross_validation.cross_val_score(lr_ridge, X_train, y_train, cv=kf)
                        predict = default_value if np.mean(scores) < 0.999 else default_value
                    except: predict = default_value
                else: predict = default_value
                #print('predict after cross validation:', predict)

            preds.append(predict)
        else:
            preds.append(default_value)
    else:
        preds.append(df1.recursion)
    
    #plt.subplot(nrows, ncols, i + 1)
    #plt.scatter(df_curr[:-1].x, df_curr[:-1].res, s=20, c='g', label='Train set ' + str(i))

    #for j in range(5):    
    #    plt.scatter(df_curr[-1:].x, df_curr[-1:].res, s=50+5**j, c='green', alpha=0.52-0.12*j, label='_')
    #    plt.scatter(df_curr[-1:].x, df_curr[-1:].predict,  s=50+5**j, c='red', alpha=0.52-0.12*j, label='_')
    #    plt.scatter(df_curr[-1:].x, df_curr[-1:].predict_ridge,  s=50+5**j, c='darkblue', alpha=0.52-0.12*j, label='_')

    #plt.plot(df_curr.x, df_curr.predict, '-', c='r', lw=0.8, ms=5, label='LinReg')
    #plt.plot(df_curr.x, df_curr.predict_ridge, '-', c='darkblue', lw=0.8, ms=5, label='LinReg Ridge')
    #plt.legend(loc='upper left', frameon=True)    
    
#print(len(df.index), len(preds))
print('First 10 predictions:', preds[:10])
print('LR time %.2f sec' % (time.time() - st))
print('Estimated time for full test data: %.2f min' % ((time.time() - st) * int(len(df.index)) / (60 * int(len(preds)))))

Sequences: 1000 of 113845 (0.88% done), time elapsed: 0.38 min, estimated time: 43.39 min
Sequences: 2000 of 113845 (1.76% done), time elapsed: 0.76 min, estimated time: 43.35 min
Sequences: 3000 of 113845 (2.64% done), time elapsed: 1.13 min, estimated time: 42.93 min
Sequences: 4000 of 113845 (3.51% done), time elapsed: 1.46 min, estimated time: 41.57 min
Sequences: 5000 of 113845 (4.39% done), time elapsed: 1.79 min, estimated time: 40.86 min
Sequences: 6000 of 113845 (5.27% done), time elapsed: 2.13 min, estimated time: 40.45 min
Sequences: 7000 of 113845 (6.15% done), time elapsed: 2.45 min, estimated time: 39.91 min
Sequences: 8000 of 113845 (7.03% done), time elapsed: 2.79 min, estimated time: 39.63 min
Sequences: 9000 of 113845 (7.91% done), time elapsed: 3.12 min, estimated time: 39.42 min
Sequences: 10000 of 113845 (8.78% done), time elapsed: 3.45 min, estimated time: 39.31 min
Sequences: 11000 of 113845 (9.66% done), time elapsed: 3.82 min, estimated time: 39.49 min
Sequence

In [62]:
df.loc[:len(preds) - 1, 'preds'] = preds
df['Last'] = df.recursion
df.preds.fillna(df.ints_list.apply(lambda x: x[-1]), inplace=True)
df.Last.fillna(df.preds, inplace=True)
df['Last'] = df['Last'].apply(lambda x: int(np.round(float(x))) if x != np.inf else 1)
print(df[['Id', 'Last']].head(10))
df[['Id', 'Last']].to_csv('submission.csv', sep=',', index=False)

   Id                        Last
0   1                           0
1   2                725161963867
2   4                         158
3   5                         406
4   6                          24
5   9                        1430
6  10                           1
7  12                           0
8  14  10576607847561078339796992
9  17      2795903786354347606016
